# Artificial Intelligence Nanodegree
## Build a Game-Playing Agent
----

## Evaluating Evaluation Functions

>내 턴에 움직일 수 있는 위치 수 - 상대방 턴에 상대방이 움직일 수 있는 위치 수

패널티를 주기 위해 상대방의 움직임에 가중치를 준다.

![isolation_1.png](images/isolation_1.png)
![isolation_2.png](images/isolation_2.png)

## Alpha-Beta Pruning

# ALPHA-BETA-SEARCH

__function__ ALPHA-BETA-SEARCH(_state_) __returns__ an action  
&emsp;_v_ &larr; MAX\-VALUE(_state_, &minus;&infin;, &plus;&infin;)  
&emsp;__return__ the _action_ in ACTIONS(_state_) with value _v_  

---
__function__ MAX\-VALUE(_state_, _&alpha;_, _&beta;_) __returns__ _a utility value_  
&emsp;__if__ TERMINAL\-TEST(_state_) __then return__ UTILITY(_state_)  
&emsp;_v_ &larr; &minus;&infin;  
&emsp;__for each__ _a_ __in__ ACTIONS(_state_) __do__  
&emsp;&emsp;&emsp;_v_ &larr; MAX(_v_, MIN\-VALUE(RESULT(_state_, _a_), _&alpha;_, _&beta;_))  
&emsp;&emsp;&emsp;__if__ _v_ &ge; _&beta;_ __then return__ _v_  
&emsp;&emsp;&emsp;_&alpha;_ &larr; MAX(_&alpha;_, _v_)  
&emsp;__return__ _v_  

---
__function__ MIN\-VALUE(_state_, _&alpha;_, _&beta;_) __returns__ _a utility value_  
&emsp;__if__ TERMINAL\-TEST(_state_) __then return__ UTILITY(_state_)  
&emsp;_v_ &larr; &plus;&infin;  
&emsp;__for each__ _a_ __in__ ACTIONS(_state_) __do__  
&emsp;&emsp;&emsp;_v_ &larr; MIN(_v_, MAX\-VALUE(RESULT(_state_, _a_), _&alpha;_, _&beta;_))  
&emsp;&emsp;&emsp;__if__ _v_ &le; _&alpha;_ __then return__ _v_  
&emsp;&emsp;&emsp;_&beta;_ &larr; MIN(_&beta;_, _v_)  
&emsp;__return__ _v_  


---
__Figure__ ?? The alpha\-beta search algorithm. Notice that these routines are the same as the MINIMAX functions in Figure ??, except for the two lines in each of MIN\-VALUE and MAX\-VALUE that maintain _&alpha;_ and _&beta;_ (and the bookkeeping to pass these parameters along).

![isolation_3.png](images/isolation_3.png)

Minimax 알고리즘으로 항상 최상의 결과를 선택할 수 있다. 하지만, 현실적으로 모든 경우의 수를 살펴 보는 것은 불가능하므로, 필요없는 부분을 생략하고 간략하게 계산을 줄일 수 있다. 각 레이어에서 Max, Min을 선택할 때 좌측 노드부터 순회 탐색하므로 특정 노드에서 값을 비교해 우측에 있는 노드들은 무시할 수 있다.

다음 노드 (Level2. 이 이미지에서 보이지 않는 옆에 있는 노드)에서는 더 많은 값들을 같은 방식으로 제거할 수 있다.

## Solving 5x5 Isolation

이런 고립된 정방향의 보드 게임에서는 축을 회전하며 생각해보면 같은 경우의 수가 여러 개 있다. 이런 게임에서 플레이어1의 승리전략은 가운데를 선점하고 플레이어 2의 위치에 미러링되는 방향으로 수를 두면 된다. 플레이어2의 승리전략은 미러링이 되지 않는 경우의 위치를 찾아 두어야 한다.

## 3-Player Games

3명이서 게임을 진행하는 경우, 각 플레이어가 선택할 수 있는 가장 큰 보상을 따라 움직이면 된다.

![isolation_4.png](images/isolation_4.png)

----
## Build a Game-Playing Agent

In [1]:
"""
Finish all TODO items in this file to complete the isolation project, then
test your agent's strength against a set of known agents using tournament.py
and include the results in your report.
"""
import random


class SearchTimeout(Exception):
    """Subclass base exception for code clarity. """
    def __init__(self):
        self.msg = "SearchTimeout Error"

    def __str__(self): #__str__ 오류메시지를 출력할 경우에 호출
        return self.msg

def custom_score(game, player):
    """
    Calculate the heuristic value of a game state from the point of view
    of the given player.

    This should be the best heuristic function for your project submission.

    Note: this function should be called from within a Player instance as
    `self.score()` -- you should not need to call this function directly.

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    player : object
        A player instance in the current game (i.e., an object corresponding to
        one of the player objects `game.__player_1__` or `game.__player_2__`.)

    Returns
    -------
    float
        The heuristic value of the current game state to the specified player.
    """
    # TODO: finish this function!
    if game.is_loser(player): #패자에게 - 보상
        return float("-inf")

    if game.is_winner(player): #승자에게 + 보상
        return float("inf")
    
    own_moves = len(game.get_legal_moves(player))
    opp_moves = len(game.get_legal_moves(game.get_opponent(player)))

    y1, x1 = game.get_player_location(player)
    y2, x2 = game.get_player_location(game.get_opponent(player))

    own_distance_from_center = math.sqrt((x1 - game.width/2)**2 + (y1 - game.height/2)**2) #중심에서 떨어진 정도 구하기
    opp_distance_from_center = math.sqrt((x2 - game.width/2)**2 + (y2 - game.height/2)**2)

    return - own_distance_from_center + opp_distance_from_center + own_moves - opp_moves


def custom_score_2(game, player):
    """
    Calculate the heuristic value of a game state from the point of view
    of the given player.

    Note: this function should be called from within a Player instance as
    `self.score()` -- you should not need to call this function directly.

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    player : object
        A player instance in the current game (i.e., an object corresponding to
        one of the player objects `game.__player_1__` or `game.__player_2__`.)

    Returns
    -------
    float
        The heuristic value of the current game state to the specified player.
    """
    # TODO: finish this function!
    if game.is_loser(player): #패자에게 - 보상
        return float("-inf")

    if game.is_winner(player): #승자에게 + 보상
        return float("inf")
    
    own_moves = len(game.get_legal_moves(player))
    opp_moves = len(game.get_legal_moves(game.get_opponent(player)))
    blank_spaces = len(game.get_blank_spaces())

    return float(blank_spaces + own_moves - opp_moves)


def custom_score_3(game, player):
    """
    Calculate the heuristic value of a game state from the point of view
    of the given player.

    Note: this function should be called from within a Player instance as
    `self.score()` -- you should not need to call this function directly.

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    player : object
        A player instance in the current game (i.e., an object corresponding to
        one of the player objects `game.__player_1__` or `game.__player_2__`.)

    Returns
    -------
    float
        The heuristic value of the current game state to the specified player.
    """
    # TODO: finish this function!
    if game.is_loser(player): #패자에게 - 보상
        return float("-inf")

    if game.is_winner(player): #승자에게 + 보상
        return float("inf")
    
    own_moves = game.get_legal_moves(player)
    opp_moves = game.get_legal_moves(game.get_opponent(player))

    similar_count = 0
    for move in opp_moves:
        if move in own_moves:
            similar_count += 1

    return float(len(own_moves) - len(opp_moves) + similar_count)


class IsolationPlayer:
    """
    Base class for minimax and alphabeta agents -- this class is never
    constructed or tested directly.

    ********************  DO NOT MODIFY THIS CLASS  ********************

    Parameters
    ----------
    search_depth : int (optional)
        A strictly positive integer (i.e., 1, 2, 3,...) for the number of
        layers in the game tree to explore for fixed-depth search. (i.e., a
        depth of one (1) would only explore the immediate sucessors of the
        current state.)

    score_fn : callable (optional)
        A function to use for heuristic evaluation of game states.

    timeout : float (optional)
        Time remaining (in milliseconds) when search is aborted. Should be a
        positive value large enough to allow the function to return before the
        timer expires.
    """
    def __init__(self, search_depth=3, score_fn=custom_score, timeout=10.):
        self.search_depth = search_depth
        self.score = score_fn
        self.time_left = None
        self.TIMER_THRESHOLD = timeout

In [ ]:
class MinimaxPlayer(IsolationPlayer):
    """
    Game-playing agent that chooses a move using depth-limited minimax
    search. You must finish and test this player to make sure it properly uses
    minimax to return a good move before the search time limit expires.
    """

    def get_move(self, game, time_left):
        """
        Search for the best move from the available legal moves and return a
        result before the time limit expires.

        **************  YOU DO NOT NEED TO MODIFY THIS FUNCTION  *************

        For fixed-depth search, this function simply wraps the call to the
        minimax method, but this method provides a common interface for all
        Isolation agents, and you will replace it in the AlphaBetaPlayer with
        iterative deepening search.

        Parameters
        ----------
        game : `isolation.Board`
            An instance of `isolation.Board` encoding the current state of the
            game (e.g., player locations and blocked cells).

        time_left : callable
            A function that returns the number of milliseconds left in the
            current turn. Returning with any less than 0 ms remaining forfeits
            the game.

        Returns
        -------
        (int, int)
            Board coordinates corresponding to a legal move; may return
            (-1, -1) if there are no available legal moves.
        """
        self.time_left = time_left

        # Initialize the best move so that this function returns something
        # in case the search fails due to timeout
        best_move = (-1, -1) #기본 값. minimax 결과로 best_move 업데이트

        try:
            # The try/except block will automatically catch the exception
            # raised when the timer is about to expire.
            return self.minimax(game, self.search_depth)

        except SearchTimeout:
            pass  # Handle any actions required after timeout as needed

        # Return the best move from the last completed search iteration
        return best_move
    
    def terminal_test(self, game):
        """ 
        Return True if the game is over for the active player
        and False otherwise.
        """
        if self.time_left() < self.TIMER_THRESHOLD:
            raise SearchTimeout()
            
        return not bool(game.get_legal_moves())
        #게임이 더 진행 될 수 있으면 False, 아니면 True.

    def min_value(self, game, depth): #상대방의 턴. 상대방은 결과가 최소(상대방 승리)가 되는 값(-1)을 찾아야 한다.
        """ 
        Return the value for a win (+1) if the game is over,
        otherwise return the minimum value over all legal child
        nodes.
        """
        if depth is 0 or self.terminal_test(game): #depth가 0이거나 게임이 종료되는 조건. min_value를 찾아야 하는 레이어(상대방의 턴)에서 게임이 종료되면 1
            return self.score(game, self) 

        v = float("inf") #무한. 어떤 값보다 크다. #import math test = math.inf
        for m in game.get_legal_moves(): #움직일 수 있는 나머지 모든 위치를 가져온다.
            v = min(v, self.max_value(game.forecast_move(m), depth-1)) #각 위치의 Max값을 가져와 비교. 최소값이 남게 된다.
            #다음 레이어(자신의 턴)에서 최대값. 이번 레이어(상대방 턴)에서 최소값.

        return v

    def max_value(self, game, depth): #자신의 턴. 자신은 결과가 최대(자신 승리)가 되는 값(1)을 찾아야 한다.
        """ 
        Return the value for a loss (-1) if the game is over,
        otherwise return the maximum value over all legal child
        nodes.
        """
        if depth is 0 or self.terminal_test(game): #depth가 0이거나 게임이 종료되는 조건. min_value를 찾아야 하는 레이어(상대방의 턴)에서 게임이 종료되면 1
            return self.score(game, self)

        v = float("-inf") #-무한. 어떤 값보다 작다.
        for m in game.get_legal_moves(): #움직일 수 있는 나머지 위치를 가져온다.
            v = max(v, self.min_value(game.forecast_move(m), depth-1)) #각 위치의 Min값을 가져와 비교. 최대값이 남게 된다.
            #다음 레이어(상대방 턴)에서 최소값. 이번 레이어(자신의 턴)에서 최대값.

        return v

    def minimax(self, game, depth):
        """
        Implement depth-limited minimax search algorithm as described in
        the lectures.

        This should be a modified version of MINIMAX-DECISION in the AIMA text.
        https://github.com/aimacode/aima-pseudocode/blob/master/md/Minimax-Decision.md

        **********************************************************************
            You MAY add additional methods to this class, or define helper
                 functions to implement the required functionality.
        **********************************************************************

        Parameters
        ----------
        game : isolation.Board
            An instance of the Isolation game `Board` class representing the
            current game state

        depth : int
            Depth is an integer representing the maximum number of plies to
            search in the game tree before aborting

        Returns
        -------
        (int, int)
            The board coordinates of the best move found in the current search;
            (-1, -1) if there are no legal moves

        Notes
        -----
            (1) You MUST use the `self.score()` method for board evaluation
                to pass the project tests; you cannot call any other evaluation
                function directly.

            (2) If you use any helper functions (e.g., as shown in the AIMA
                pseudocode) then you must copy the timer check into the top of
                each helper function or else your agent will timeout during
                testing.
        """
#         if self.time_left() < self.TIMER_THRESHOLD:
#             raise SearchTimeout()
        # TODO: finish this function!
            
        best_score = float("-inf") 
        best_move = (-1, -1)

        for m in game.get_legal_moves(): #이동할 수 있는 모든 곳 중에서
            v = self.min_value(game.forecast_move(m), depth-1) #max가 되는 값을 찾아야 하므로 다음 레이어의 min_value를 찾아야 한다.
            if v > best_score:
                best_score = v
                best_move = m

        return best_move

In [2]:
class AlphaBetaPlayer(IsolationPlayer):
    """
    Game-playing agent that chooses a move using iterative deepening minimax
    search with alpha-beta pruning. You must finish and test this player to
    make sure it returns a good move before the search time limit expires.
    """

    def get_move(self, game, time_left):
        """
        Search for the best move from the available legal moves and return a
        result before the time limit expires.

        Modify the get_move() method from the MinimaxPlayer class to implement
        iterative deepening search instead of fixed-depth search.

        **********************************************************************
        NOTE: If time_left() < 0 when this function returns, the agent will
              forfeit the game due to timeout. You must return _before_ the
              timer reaches 0.
        **********************************************************************

        Parameters
        ----------
        game : `isolation.Board`
            An instance of `isolation.Board` encoding the current state of the
            game (e.g., player locations and blocked cells).

        time_left : callable
            A function that returns the number of milliseconds left in the
            current turn. Returning with any less than 0 ms remaining forfeits
            the game.

        Returns
        -------
        (int, int)
            Board coordinates corresponding to a legal move; may return
            (-1, -1) if there are no available legal moves.
        """
        self.time_left = time_left
        
        # TODO: finish this function!
        # Initialize the best move so that this function returns something
        # in case the search fails due to timeout
        best_move = (-1, -1) #기본 값. minimax 결과로 best_move 업데이트

        try:
            # The try/except block will automatically catch the exception
            # raised when the timer is about to expire.
            depth = 1

            while True:
                best_move = self.alphabeta(game, depth) #depth를 높여가면서 탐색해 값을 찾는다.
                depth+=1

        except SearchTimeout:
            pass  # Handle any actions required after timeout as needed

        # Return the best move from the last completed search iteration
        return best_move
    
    def terminal_test(self, game):
        """ 
        Return True if the game is over for the active player
        and False otherwise.
        """
        if self.time_left() < self.TIMER_THRESHOLD:
            raise SearchTimeout()
            
        return not bool(game.get_legal_moves())
        #게임이 더 진행 될 수 있으면 False, 아니면 True.

    def min_value(self, game, depth, alpha, beta): #상대방의 턴. 상대방은 결과가 최소(상대방 승리)가 되는 값(-1)을 찾아야 한다.
        """ 
        Return the value for a win (+1) if the game is over,
        otherwise return the minimum value over all legal child
        nodes.
        """
        if depth is 0 or self.terminal_test(game): #depth가 0이거나 게임이 종료되는 조건. min_value를 찾아야 하는 레이어(상대방의 턴)에서 게임이 종료되면 1
            return self.score(game, self) 

        v = float("inf") #무한. 어떤 값보다 크다. #import math test = math.inf
        for m in game.get_legal_moves(): #움직일 수 있는 나머지 모든 위치를 가져온다.
            v = min(v, self.max_value(game.forecast_move(m), depth-1, alpha, beta)) #각 위치의 Max값을 가져와 비교. 최소값이 남게 된다.
            #다음 레이어(자신의 턴)에서 최대값. 이번 레이어(상대방 턴)에서 최소값.
            
            if v <= alpha: #가지치기
                return v
            beta = min(beta, v)

        return v

    def max_value(self, game, depth, alpha, beta): #자신의 턴. 자신은 결과가 최대(자신 승리)가 되는 값(1)을 찾아야 한다.
        """ 
        Return the value for a loss (-1) if the game is over,
        otherwise return the maximum value over all legal child
        nodes.
        """
        if depth is 0 or self.terminal_test(game): #depth가 0이거나 게임이 종료되는 조건. min_value를 찾아야 하는 레이어(상대방의 턴)에서 게임이 종료되면 1
            return self.score(game, self)

        v = float("-inf") #-무한. 어떤 값보다 작다.
        for m in game.get_legal_moves(): #움직일 수 있는 나머지 위치를 가져온다.
            v = max(v, self.min_value(game.forecast_move(m), depth-1, alpha, beta)) #각 위치의 Min값을 가져와 비교. 최대값이 남게 된다.
            #다음 레이어(상대방 턴)에서 최소값. 이번 레이어(자신의 턴)에서 최대값.
            
            if v >= beta: #가지치기
                return v
            alpha = max(alpha, v)

        return v

    def alphabeta(self, game, depth, alpha=float("-inf"), beta=float("inf")):
        """
        Implement depth-limited minimax search with alpha-beta pruning as
        described in the lectures.

        This should be a modified version of ALPHA-BETA-SEARCH in the AIMA text
        https://github.com/aimacode/aima-pseudocode/blob/master/md/Alpha-Beta-Search.md

        **********************************************************************
            You MAY add additional methods to this class, or define helper
                 functions to implement the required functionality.
        **********************************************************************

        Parameters
        ----------
        game : isolation.Board
            An instance of the Isolation game `Board` class representing the
            current game state

        depth : int
            Depth is an integer representing the maximum number of plies to
            search in the game tree before aborting

        alpha : float
            Alpha limits the lower bound of search on minimizing layers

        beta : float
            Beta limits the upper bound of search on maximizing layers

        Returns
        -------
        (int, int)
            The board coordinates of the best move found in the current search;
            (-1, -1) if there are no legal moves

        Notes
        -----
            (1) You MUST use the `self.score()` method for board evaluation
                to pass the project tests; you cannot call any other evaluation
                function directly.

            (2) If you use any helper functions (e.g., as shown in the AIMA
                pseudocode) then you must copy the timer check into the top of
                each helper function or else your agent will timeout during
                testing.
        """
#         if self.time_left() < self.TIMER_THRESHOLD:
#             raise SearchTimeout()

        # TODO: finish this function!
        best_score = float("-inf") 
        best_move = (-1, -1)

        for m in game.get_legal_moves(): #이동할 수 있는 모든 곳 중에서
            v = self.min_value(game.forecast_move(m), depth-1, alpha, beta) #max가 되는 값을 찾아야 하므로 다음 레이어의 min_value를 찾아야 한다.
            if v > best_score:
                best_score = v
                best_move = m
                
            alpha = max(alpha, v) #alpha는 가지 쳐내기 위한 max

        return best_move